## Tutorial 19 - Interpolation & Integration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from scipy import integrate

### Numerical integration

Recall that in Homework 4, we used a numerical integration method to calculate the number of detections we expect over 10 seconds given the Detector Response Function we were given. This was also described in the slides for Lecture 12. 

The method that we used at the time was actually a midpoint rule integration. 

\begin{align*}
    N_\mathrm{det} &= T_\mathrm{obs} \int^{E_1}_{E_0} f_E(E)\ A_\mathrm{eff}(E)\ dE \qquad &&\text{integral form}\\
    N_\mathrm{det} &= T_\mathrm{obs} \sum_i f_E(E_i^{\ \mathrm{mid}})\ A_\mathrm{eff}(E_i^{\ \mathrm{mid}})\ \Delta E \qquad &&\text{sum approximation}\\
\end{align*}
where  $\Delta E = E_{i+1} - E_i$  and  $E_i^{\ \mathrm{mid}} = (E_i + E_{i+1})/2$.

* Download the fits file that we used in Homework 4 again that contains the detector response matrix (Can be found on Canvas [here](https://psu.instructure.com/courses/2367528/files/174931328?wrap=1)).

* Get the HDU with the DRM in it. Remember this will be a table with a column named MATRIX in it. This column is the DRM.
* The other columns in this table are named ENERG_LO and ENERG_HI, these are the low and high photon energies that each row in the DRM corresponds to. 
    - Find the midpoint between ENERG_LO and ENERG_HI and set that array equal to a variable named `Photon_Energies`

In [ ]:
f = fits.open('Fermi_GBM_DRM.fits')

drm = BLANK
Photon_Energies  = BLANK

The function below should be similar to the one you wrote in Homework 4. It calculates the differential photon flux for a power-law spectrum at an array of photon energies. 

The inputs are:
* an array of photon energies
* the spectral index, $\alpha$
* the normalization factor, $C$

In [ ]:
def f_E(E, alpha, C):
    '''
    Compute the differential photon flux for a power-law spectrum

    Parameters:
    -----------
    E: array
        Photon energies in keV
    alpha: float
        Spectral index 
    C: float
        Normalization factor in photons / cm^2 / s / keV

    Returns:
    -----------
    array 
        Differential photon flux 

    '''

    E0 = 100                        # pivot energy = 100 keV
    f_E = C * (E/E0) ** alpha       # calculate f_E

    return f_E

* Find the effective area from the DRM as a function of photon energy
    - Hint: We did this in Tutorial 10 as well as Homework 4
* Using your function calculate the differential photon fluxes at the energies in Photon_Energies for $\alpha = -2$, and $C = 5 \times 10^{-4}$ photons / cm $^2$ / s / keV

In [ ]:
Aeff = BLANK
fluxes = BLANK

Use the trapezoid rule + simpsons rule functions in `scipy.integrate` to find the # of detections in a 10 second exposure.
* `integrate.trapezoid(y, x)`
* `integrate.simpsons(y, x)`

In [ ]:
# Trapezoid rule


In [ ]:
# Simpsons rule

### Interpolation

For the other function in `scipy.intergrate` (i.e. `quad`), we need the values we are integrating to have a "functional" form (similar to `minimize`). This is an area where we can use our interpolation functions in `scipy.interpolate`

In [ ]:
from scipy import interpolate

Plot the "integrand" of the integral we are computing $f_E(E)\ A_\mathrm{eff}(E)$. Use circles for your markers.

In [ ]:
plt.plot(Photon_Energies, fluxes*Aeff , 'o')
plt.semilogx()                                  # set the x-axis to be log-scaled

# add labels to your plot

We can see here that our "x" values are not evenly spaced in linear space. Rather they are evenly spaced on a logarithmic scale. Because our interpolate functions want evenly spaced values, we can define a new variable `log_Photon_Energies` which we'll use for this part in place of `Photon_Energies`

In [ ]:
log_Photon_Energies = np.log10(Photon_Energies)
plt.plot(log_Photon_Energies, fluxes*Aeff, 'o')

# add labels to your plot

First, use `np.interp1d` to get a linear interpolation of the points.

In [ ]:
# Linear interpolation

xnew = np.linspace(np.min(log_Photon_Energies), np.max(log_Photon_Energies), 1000)
linear = BLANK

* Use `interpolate.interp1d` to get a piece-wise interpolation of the points
    - Note: reminder to use the keyword `kind="nearest"` for this
* Use `interpolate.CubicSpline` to get a Cubic Spline

**Remember** these functions each return a **function** that takes an array of $x$ values and returns the "interpolated" $y$-values

In [ ]:
# Piecewise
piecewise_f = BLANK

In [ ]:
# Cubic
cubic_f = BLANK

Plot each of these functions between `np.min(log_Photon_Energies)` and `np.max(log_Photon_Energies)`

In [ ]:
plt.plot(log_Photon_Energies, fluxes*Aeff, 'o')
plt.plot(xnew, piecewise_f(xnew), label='Piecewise')
plt.plot(xnew, linear, label='Linear')
plt.plot(xnew, cubic_f(xnew), label='Cubic')
plt.legend()

In [ ]:
# If we zoom in using plt.xlim we can see the difference between the three methods

plt.plot(log_Photon_Energies, fluxes*Aeff, 'o')
plt.plot(xnew, piecewise_f(xnew), label='Piecewise')
plt.plot(xnew, linear, label='Linear')
plt.plot(xnew, cubic_f(xnew), '--', label='Cubic')
plt.legend()
plt.xlim(1,1.4)
plt.ylim(0.75,1.4)

Now we can use `integrate.quad` on each of these functions. 
Important inputs:
* `func` - function
* `a` - lower bound of integral
* `b` - upper bound of integral

Use `integrate.quad(func, a = np.min(Photon_Energies), b = np.max(Photon_Energies))`

In [ ]:
integrate.quad?

Calculate $N_\mathrm{det}$ using both interpolation methods for an exposure time of 10 seconds. 

First, run the following code to get new versions of the piecewise interpolation function and cubic spline interpolation function that will take Photon_Energies as input instead of log_Photon_Energies.
You will use these functions as your "integrand".

In [ ]:
def piecewise_rescaled(E):
    log_E = np.log10(E)
    return piecewise_f(log_E)

def cubic_rescaled(E):
    log_E = np.log10(E)
    return cubic_f(log_E)

In [ ]:
# Integrate the piecewise function "piecewise_rescaled" using the quad function
Ndet_1 = BLANK

# Repeat for the cubic
Ndet_2 = BLANK

# print your results for both of these methods